# Recommendation System Datasets

This notebook uses the following datasets:

- [MovieLens 10M data set](http://grouplens.org/datasets/movielens/10m/)
- [MovieLens 22M data set](http://grouplens.org/datasets/movielens/latest/)
- [Million song data set](http://labrosa.ee.columbia.edu/millionsong/tasteprofile)

## Split dataset into 60-20-20 train-validate-test partitions

In [4]:
import os

def exists(filepath):
    return os.path.exists(filepath)

In [5]:
if (exists('ml-10M100K/train60.dat') and exists('ml-10M100K/validation20.dat') and exists('ml-10M100K/test20.dat')):
    print "Already created files: train60.dat, validation20.dat, test20.dat"    

else:
    # sort by timestamp (4th column)
    print 'sorting file...'
    !sort -t ':' -k4 ml-10M100K/ratings.dat > ml-10M100K/new_ratings.dat 
    print "sorting complete."
    
    # split into 5 parts of 2 million each: train(3 parts), validation (1 part), test (1 part)
    print "splitting file..."
    !split -l 2000000 ml-10M100K/new_ratings.dat ff
    !cat ffaa ffab ffac > ml-10M100K/train60.dat
    !mv ffad ml-10M100K/validation20.dat
    !mv ffae ml-10M100K/test20.dat
    
    # remove tmp files used to create partitions
    !rm new_ratings.dat ff*
    print "splitting complete."    
    print "Newly created files: train60.dat, validation20.dat, test20.dat"

Already created files: train60.dat, validation20.dat, test20.dat


# Using train data, learn ALS model

In [82]:
import contextlib
from math import sqrt
from operator import add
import sys
from pyspark.mllib.recommendation import ALS

In [83]:
def parse_rating(line):
    """
    Parses a rating record that's in MovieLens format.
    
    :param str line: userId::movieId::rating::timestamp
    """
    fields = line.strip().split("::")

    return (int(fields[0]),   # User ID
            int(fields[1]),   # Movie ID
            float(fields[2])) # Rating


def compute_rmse(model, data, validationCount):
    """
    Compute RMSE (Root Mean Squared Error).
    :param object model
    :param list data
    :param integer validation_count
    """
    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    predictionsAndRatings = \
        predictions.map(lambda x: ((x[0], x[1]), x[2])) \
                   .join(data.map(lambda x: ((x[0], x[1]), x[2]))) \
                   .values()
    return sqrt(
        predictionsAndRatings.map(
            lambda x: (x[0] - x[1]) ** 2
        ).reduce(add) / float(validationCount)
    )

In [35]:
training = sc.textFile('ml-10M100K/train60.dat') \
         .filter(lambda x: x and len(x.split('::')) == 4) \
         .map(parse_rating)

In [36]:
validation = sc.textFile('ml-10M100K/validation20.dat') \
         .filter(lambda x: x and len(x.split('::')) == 4) \
         .map(parse_rating)

In [37]:
test = sc.textFile('ml-10M100K/test20.dat') \
         .filter(lambda x: x and len(x.split('::')) == 4) \
         .map(parse_rating)

In [38]:
trainCount = training.count()
trainCount

6000000

In [39]:
validationCount = validation.count()
validationCount

2000000

In [40]:
testCount = test.count()
testCount

2000000

### Train ALS model using different regularization parameter and latent factors

In [41]:
rank_list = [10, 20, 30, 40, 50] # latent factor
lamda_list = [0.01, 0.1, 1.0, 10.0] # regularization parameter
iterations = 10
chosenModel = None
smallestRMSE = 9999999

for rank in rank_list:
    for lamda in lamda_list:
        model = ALS.train(training, rank, iterations, lamda)
        rmse = compute_rmse(model, validation, validationCount)
        
        if rmse < smallestRMSE:
            smallestRMSE = rmse
            chosenModel = model

        print 'Rank={}, Lambda={}, RMSE={}'.format(rank, lamda, rmse)

Rank=10, Lambda=0.01, RMSE=1.06049775091
Rank=10, Lambda=0.1, RMSE=1.10979440812
Rank=10, Lambda=1.0, RMSE=1.95801505852
Rank=10, Lambda=10.0, RMSE=3.99960898089
Rank=20, Lambda=0.01, RMSE=1.07348413238
Rank=20, Lambda=0.1, RMSE=1.14287691247
Rank=20, Lambda=1.0, RMSE=1.95987810731
Rank=20, Lambda=10.0, RMSE=3.99960898089
Rank=30, Lambda=0.01, RMSE=1.08488028136
Rank=30, Lambda=0.1, RMSE=1.1275414034
Rank=30, Lambda=1.0, RMSE=1.95889910186
Rank=30, Lambda=10.0, RMSE=3.99960898089
Rank=40, Lambda=0.01, RMSE=1.08504936949
Rank=40, Lambda=0.1, RMSE=1.16572274978
Rank=40, Lambda=1.0, RMSE=1.95928678985
Rank=40, Lambda=10.0, RMSE=3.99960898089
Rank=50, Lambda=0.01, RMSE=1.10281105899
Rank=50, Lambda=0.1, RMSE=1.18677347337
Rank=50, Lambda=1.0, RMSE=1.95958789238
Rank=50, Lambda=10.0, RMSE=3.99960898089


In [61]:
chosenModel.save(sc, 'chosenModel')

In [63]:
print 'The smallest RMSE is:{0: .2f}'.format(smallestRMSE)

The smallest RMSE is: 1.06


### Use chosen model with test set

In [60]:
testRMSE = compute_rmse(chosenModel, test, testCount)
print 'Final error metric using test set ={0: .2f}'.format(testRMSE)

Final error metric using test set = 1.87


In [ ]:
def generate_recommendations(model, ratingsFile):
    prediction = []
    return prediction

In [73]:
ratings = sc.textFile('ml-10M100K/ratings.dat') \
         .filter(lambda x: x and len(x.split('::')) == 4) \
         .map(parse_rating)

In [74]:
ratings.take(3) # userID::movieID::ratings

[(1, 122, 5.0), (1, 185, 5.0), (1, 231, 5.0)]

In [78]:
userRatings = ratings.filter(lambda x: x[0] == 1) # get only movie ratings for user with userid=1
userRatings.count()

0

In [81]:
userRatings

PythonRDD[2899] at RDD at PythonRDD.scala:43

In [75]:
ratings = sc.textFile('ml-10M100K/ratings.dat')
ratings.take(3)

[u'1::122::5::838985046', u'1::185::5::838983525', u'1::231::5::838983392']

In [88]:
help(ALS.train)

Help on method train in module pyspark.mllib.recommendation:

train(cls, ratings, rank, iterations=5, lambda_=0.01, blocks=-1, nonnegative=False, seed=None) method of __builtin__.type instance



### Meaning of parameters

- numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
- ***rank*** is the number of latent factors in the model.
- iterations is the number of iterations to run.
- ***lambda*** specifies the regularization parameter in ALS.
- implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
- alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.


In [89]:
training.take(3)

[(37746, 3409, 0.5), (37746, 175, 0.5), (51778, 5430, 0.5)]

In [94]:
validation.take(3)

[(6352, 6787, 4.0), (26571, 1580, 4.0), (26571, 2115, 4.0)]

In [95]:
test.take(3)

[(5337, 296, 4.0), (5337, 307, 4.0), (32329, 3745, 4.0)]

### Train and Save ALS Model

In [91]:
model = ALS.train(training, 10, 20, 0.01)

In [92]:
model.save(sc, 'myALSmodel')

### Try model with validation set (user-product pairs needed)

In [97]:
help(model.predictAll)

Help on method predictAll in module pyspark.mllib.recommendation:

predictAll(self, user_product) method of pyspark.mllib.recommendation.MatrixFactorizationModel instance
    Returns a list of predicted ratings for input user and product pairs.



In [98]:
validation_pair = validation.map(lambda x: (x[0], x[1]))
validation_pair.take(3)

[(6352, 6787), (26571, 1580), (26571, 2115)]

In [99]:
predictions = model.predictAll(validation_pair)
predictions.take(3)

[Rating(user=57436, product=1356, rating=2.9622857548829815),
 Rating(user=57436, product=648, rating=2.7055366122842517),
 Rating(user=57436, product=260, rating=2.618657352015365)]

In [100]:
predictions = predictions.map(lambda x: (x[0], x[1], x[2]))
predictions.take(3)

[(57436, 1356, 2.9622857548829815),
 (57436, 648, 2.7055366122842517),
 (57436, 260, 2.618657352015365)]